<a href="https://colab.research.google.com/github/maevezhw/web-scraping/blob/main/Data%20Terstruktur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Library

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs4

Scrape from Website

In [ ]:
countries_population_page = requests.get("https://www.worldometers.info/world-population/population-by-country/").text
countries_population_soup = bs4(countries_population_page, "html.parser")
countries_population_table = countries_population_soup.find_all('table', {'id' : "example2"})

In [ ]:
# membaca table dari html dan mengubahnya menjadi list dataframe
countries_population_table_df = pd.read_html(str(countries_population_table))

# mengubah list dataframe menjadi dataframe
countries_population_table_df = countries_population_table_df[0]

# merapikan kolom
countries_population_table_df = countries_population_table_df.rename(columns = {'#': "Rank", "Country (or dependency)": "Country"})
countries_population_table_df.drop(columns = {"Urban Pop %", "World Share"}, inplace= True)

# memanggil dataframe
countries_population_table_df

,Rank,Country,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age
0,1,China,1439323776,0.39 %,5540090,153,9388211,-348399.0,1.7,38
1,2,India,1380004385,0.99 %,13586631,464,2973190,-532687.0,2.2,28
2,3,United States,331002651,0.59 %,1937734,36,9147420,954806.0,1.8,38
3,4,Indonesia,273523615,1.07 %,2898047,151,1811570,-98955.0,2.3,30
4,5,Pakistan,220892340,2.00 %,4327022,287,770880,-233379.0,3.6,23
...,...,...,...,...,...,...,...,...,...,...
230,231,Montserrat,4992,0.06 %,3,50,100,NaN,N.A.,N.A.
231,232,Falkland Islands,3480,3.05 %,103,0,12170,NaN,N.A.,N.A.
232,233,Niue,1626,0.68 %,11,6,260,NaN,N.A.,N.A.
233,234,Tokelau,1357,1.27 %,17,136,10,NaN,N.A.,N.A.


Preprocessing

In [ ]:
# mengubah format penulisan yang masih salah (N.A. tidak terdeteksi sebagai nilai kosong oleh Python, sehingga harus diubah dulu)
countries_population_table_df.replace("N.A.", np.nan, inplace=True)
countries_population_table_df.replace("NaN", np.nan, inplace=True)

In [ ]:
# menggunakan interpolate untuk memperkirakan nilai data dan memasukkannya ke bagian kolom yang kosong
countries_population_table_df["Migrants (net)"].interpolate(inplace=True)

# menggunakan fungsi fillna untuk mengisi nilai pada kolom yang kosong dengan nilai rata-rata dari kolom tersebut
countries_population_table_df["Fert. Rate"].fillna(pd.to_numeric(countries_population_table_df["Fert. Rate"]).mean(), inplace=True)
countries_population_table_df["Med. Age"].fillna(pd.to_numeric(countries_population_table_df["Med. Age"]).mean(), inplace=True)

# mengubah format nilai menjadi numerik agar bisa dilakukan operasi numerik
countries_population_table_df["Fert. Rate"] = pd.to_numeric(countries_population_table_df["Fert. Rate"])
countries_population_table_df["Med. Age"] = pd.to_numeric(countries_population_table_df["Med. Age"])

countries_population_table_df

,Rank,Country,Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age
0,1,China,1439323776,0.39 %,5540090,153,9388211,-348399.0,1.70000,38.000000
1,2,India,1380004385,0.99 %,13586631,464,2973190,-532687.0,2.20000,28.000000
2,3,United States,331002651,0.59 %,1937734,36,9147420,954806.0,1.80000,38.000000
3,4,Indonesia,273523615,1.07 %,2898047,151,1811570,-98955.0,2.30000,30.000000
4,5,Pakistan,220892340,2.00 %,4327022,287,770880,-233379.0,3.60000,23.000000
...,...,...,...,...,...,...,...,...,...,...
230,231,Montserrat,4992,0.06 %,3,50,100,0.0,2.69204,30.621891
231,232,Falkland Islands,3480,3.05 %,103,0,12170,0.0,2.69204,30.621891
232,233,Niue,1626,0.68 %,11,6,260,0.0,2.69204,30.621891
233,234,Tokelau,1357,1.27 %,17,136,10,0.0,2.69204,30.621891


In [ ]:
countries_population_table_df.describe()

,Rank,Population (2020),Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age
count,235.000000,2.350000e+02,2.350000e+02,235.000000,2.350000e+02,235.000000,235.000000,235.000000
mean,118.000000,3.317120e+07,3.461058e+05,478.408511,5.535918e+05,-4.655319,2.692040,30.621891
std,67.982841,1.351370e+08,1.128255e+06,2331.282424,1.687796e+06,113983.547578,1.168803,8.431033
min,1.000000,8.010000e+02,-3.838400e+05,0.000000,0.000000e+00,-653249.000000,1.100000,15.000000
25%,59.500000,4.188015e+05,4.240000e+02,37.000000,2.545000e+03,-8608.000000,1.800000,24.000000
50%,118.000000,5.459642e+06,3.917000e+04,95.000000,7.724000e+04,0.000000,2.400000,30.621891
75%,176.500000,2.057705e+07,2.496600e+05,240.000000,4.038200e+05,4900.000000,3.050000,37.500000
max,235.000000,1.439324e+09,1.358663e+07,26337.000000,1.637687e+07,954806.000000,7.000000,48.000000


In [ ]:
from google.colab import files
countries_population_table_df.to_csv('countries_population.csv')
files.download('countries_population.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>